In [1]:
from glob import glob
import pandas as pd
import re
from pprint import pprint
import pkg_resources

from pymedextcore.document import Document
from pymedext_eds.annotators import Endlines, SentenceTokenizer, SectionSplitter
from pymedext_eds.utils import rawtext_loader
from pymedext_eds.med import MedicationAnnotator, MedicationNormalizer

In [2]:
models_param = [{'tagger_path':'/export/home/cse180025/prod_information_extraction/data/models/apmed4/entities/final-model.pt' ,
                'tag_name': 'entity_pred' },
                {'tagger_path':'/export/home/cse180025/prod_information_extraction/data/models/apmed4/events/final-model.pt' ,
                'tag_name': 'event_pred' },
               {'tagger_path': "/export/home/cse180025/prod_information_extraction/data/models/apmed4/drugblob/final-model.pt",
                'tag_name': 'drugblob_pred'}]

data_path = pkg_resources.resource_filename('pymedext_eds', 'data/romedi')
romedi_path = glob(data_path + '/*.p')[0]

In [3]:
models_param = [{'tagger_path': 'fr-ner',
                'tag_name': 'entity_pred' }]

data_path = pkg_resources.resource_filename('pymedext_eds', 'data/romedi')
romedi_path = glob(data_path + '/*.p')[0]

In [4]:
endlines = Endlines(["raw_text"], "clean_text", ID="endlines")
sections = SectionSplitter(['clean_text'], "section", ID= 'sections')
sentenceSplitter = SentenceTokenizer(["section"],"sentence", ID="sentences")
med = MedicationAnnotator(['sentence'], 'med', ID='med:v2', models_param=models_param,  device='cpu')
norm = MedicationNormalizer(['ENT/DRUG','ENT/CLASS'], 'normalized_mention', ID='norm',romedi_path= romedi_path)

pipeline = [endlines,sections, sentenceSplitter, med, norm]

2020-12-23 16:52:04,827 loading file /home/basile/.flair/models/fr-ner-wikiner-0.4.pt


In [5]:
data_path = pkg_resources.resource_filename('pymedext_eds', 'data/demo')
file_list = glob(data_path + '/*.txt')

docs = [rawtext_loader(x) for x in file_list]

In [6]:
for doc in docs:
    doc.annotate(pipeline)

Ignore 2 sentence(s) with no tokens.


In [7]:
med.annotate_function(doc)

[]

In [8]:
model = med.model_zoo[0][1]

In [9]:
from flair.data import Sentence

In [10]:
model.predict

<bound method SequenceTagger.predict of SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings('fr')
    (list_embedding_1): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.5, inplace=False)
        (encoder): Embedding(275, 100)
        (rnn): LSTM(100, 1024)
        (decoder): Linear(in_features=1024, out_features=275, bias=True)
      )
    )
    (list_embedding_2): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.5, inplace=False)
        (encoder): Embedding(275, 100)
        (rnn): LSTM(100, 1024)
        (decoder): Linear(in_features=1024, out_features=275, bias=True)
      )
    )
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=2348, out_features=2348, bias=True)
  (rnn): LSTM(2348, 256, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=512, out_features=20, bias=True)
)>

In [11]:
s = Sentence('George Washington est allé à Washington.')

In [12]:
model.predict(s)

In [13]:
s.to_dict()

{'text': 'George Washington est allé à Washington.',
 'labels': [],
 'entities': []}

In [14]:
print(s.to_tagged_string())

George <B-PER> Washington <E-PER> est allé à Washington <S-LOC> .


In [15]:
doc.get_annotations('clean_text')[0].to_dict()

{'type': 'clean_text',
 'value': 'Antécédents médicaux 1,62 m pour 65,7 kg = IMC à 25,03.. En 1974 ,19 ans , , embolie pulmonaire compliquant une phlébite lors d\'un alitement à la suite d’un accident de la voie publique. Des bilans d\'hémostase effectués en 1995 et 2004 avaient éliminé une anomalie congénitale de la coagulation.. Insuffisance veineuse saphène interne bilatérale connue depuis 6 ans pour laquelle avait été proposée une intervention.. Mode de vie. Autonome et indépendante dans tous les gestes de la vie courante.. Loisirs : course à pied, semi-marathon, randonnées dans le cadre d\'un club ainsi que pratique du vélo.. Le chirurgien conclue :. "Gonalgies droites évoluant depuis novembre 2012. Très sportive, a dû arrêter à cause de ses douleurs. Pas de blocage, ni d\'instabilité. Escaliers difficiles. Réveil nocturne. A eu plusieurs infiltrations d’acide hyaluronique sans aucun effet.. Examen :. Bonnes amplitudes articulaires. Choc rotulien. Ligaments OK. Rotule OK. Douleurs

In [23]:
i = 10

In [70]:
i += 1
docs[0].get_annotations('sentence')[i].to_dict()

{'type': 'sentence',
 'value': ' Le courrier de sortie de réanimation conclue à : "Etat de choc mixte, d\'une part choc anaphylactique à la Pyostacine® et d\'autre part choc septique en rapport avec des abcès pulmonaires dont la porte d\'entrée est probablement cutanée',
 'ngram': None,
 'span': (5026, 5260),
 'source': 'sentences',
 'source_ID': 'd9cdb265-4536-11eb-864c-3b15d0b9c6c7',
 'isEntity': False,
 'attributes': {'section': 'head'},
 'ID': 'd9cdb29f-4536-11eb-864c-3b15d0b9c6c7'}

In [7]:
pd.DataFrame.from_records(MedicationAnnotator.doc_to_omop(docs[1]))

""
